In [454]:
from lxml import etree, html
from lxml.html.clean import clean_html
import pandas as pd
from datetime import date, datetime
import requests
import re
import os
import shutil

# Mueve todos los fichero del directorio de trabajo a otro de archivo

In [456]:
source_dir = './BOEs'
target_dir = './BOEs_Anteriores'
    
file_names = os.listdir(source_dir)
    
for file_name in file_names:
    #shutil.move(os.path.join(source_dir, file_name), target_dir)
    # Evita que de error si el fichero que se mueve ya existe en dir destino
    try:
        os.remove(os.path.join(target_dir, file_name))
        shutil.move(os.path.join(source_dir, file_name), target_dir)
    except OSError:
        shutil.move(os.path.join(source_dir, file_name), target_dir)

# Recoge resumen diario del BOE de hoy

In [699]:

today = date.today()

# dd/mm/YYYY
hoy = today.strftime("%Y%m%d")
print("Fecha de Hoy =", hoy)

# dd
d = today.strftime("%d")
print("dia =", d)

# mm
m = today.strftime("%m")
print("mes =", m)

# YYYY
Y = today.strftime("%Y")
print("año =", Y)

print(today.strftime("%d/%m/%Y"))

Fecha de Hoy = 20210307
dia = 07
mes = 03
año = 2021
07/03/2021


In [700]:
URL_XML_resumen =  "https://www.boe.es/diario_boe/xml.php?id=BOE-S-" + str(hoy)

In [701]:
# URL_XML_resumen = 'https://www.boe.es/diario_boe/xml.php?id=BOE-S-20210223' # Contiene Seccion T. Constitucional
# URL_XML_resumen = 'https://www.boe.es/diario_boe/xml.php?id=BOE-S-20210305' # Contiene Sección V. A

In [702]:
URL_XML_resumen

'https://www.boe.es/diario_boe/xml.php?id=BOE-S-20210305'

In [703]:
url = URL_XML_resumen
r = requests.get(url)

In [704]:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)

save_html(r.content, './BOEs/Resumen-BOE-' + hoy + '.xml')

In [705]:
resumen = etree.parse('./BOEs/Resumen-BOE-' + hoy + '.xml')

In [706]:
raiz=resumen.getroot()
raiz.tag
raiz_sumario = raiz

In [707]:
seccion = raiz.findall("sumario/diario/seccion")
seccion

[]

In [708]:
for seccion in raiz.xpath('//seccion'):
    nombre_seccion = seccion.xpath('@nombre')
    print(nombre_seccion)

['I. Disposiciones generales']
['II. Autoridades y personal. - A. Nombramientos, situaciones e incidencias']
['II. Autoridades y personal. - B. Oposiciones y concursos']
['III. Otras disposiciones']
['IV. Administración de Justicia']
['V. Anuncios. - A. Contratación del Sector Público']
['V. Anuncios. - B. Otros anuncios oficiales']


In [709]:
secciones = raiz.findall("sumario/diario/seccion")
for seccion in secciones:
    print(seccion.text)

In [710]:
tabla_resumen = pd.DataFrame()

In [711]:
for secciones in raiz.xpath('//seccion[contains(@nombre, "I. Disposiciones generales") or contains(@nombre, "III. Otras disposiciones") or contains(@nombre, "V. Anuncios. - A. Contratación del Sector Público") or contains(@nombre, "T.C. Sección del Tribunal Constitucional")]'):

    nombre_seccion = secciones.xpath('@nombre')
    print(nombre_seccion)

    for seccion in secciones:

        for item in seccion.xpath('.//item'):
            item_id = item.xpath('@id')
            print(item_id)
            nombre_departamento = item.xpath('.//ancestor::departamento/@nombre')
            print(nombre_departamento)
            nombre_epigrafe = item.xpath('.//ancestor::epigrafe/@nombre')
            print(nombre_epigrafe)
            item_name = item.xpath('.//titulo/text()')
            item_urlXml = "https://www.boe.es" + str(item.xpath('.//urlXml/text()'))[2:-2]
            print(item_urlXml)
            tabla_resumen = tabla_resumen.append({'Seccion': nombre_seccion, 
                                                  'Departamento': nombre_departamento, 
                                                  'Epigrafe' : nombre_epigrafe,
                                                  'Item_id': item_id, 
                                                  'Item_Nombre' : item_name, 
                                                  'Item_URL_XML' : item_urlXml},
                                                  ignore_index=True)

['I. Disposiciones generales']
['BOE-A-2021-3417']
['MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTACIÓN']
['Fondos europeos agrícolas']
https://www.boe.es/diario_boe/xml.php?id=BOE-A-2021-3417
['BOE-A-2021-3418']
['MINISTERIO PARA LA TRANSICIÓN ECOLÓGICA Y EL RETO DEMOGRÁFICO']
['Combustibles. Precios']
https://www.boe.es/diario_boe/xml.php?id=BOE-A-2021-3418
['BOE-A-2021-3419']
['MINISTERIO DE ASUNTOS ECONÓMICOS Y TRANSFORMACIÓN DIGITAL']
['Ordenación de pagos']
https://www.boe.es/diario_boe/xml.php?id=BOE-A-2021-3419
['BOE-A-2021-3420']
['MINISTERIO DE ASUNTOS ECONÓMICOS Y TRANSFORMACIÓN DIGITAL']
['Medidas financieras']
https://www.boe.es/diario_boe/xml.php?id=BOE-A-2021-3420
['BOE-A-2021-3421']
['MINISTERIO DE INCLUSIÓN, SEGURIDAD SOCIAL Y MIGRACIONES']
['Procedimientos administrativos. Gestión informatizada']
https://www.boe.es/diario_boe/xml.php?id=BOE-A-2021-3421
['III. Otras disposiciones']
['BOE-A-2021-3440']
['CONSEJO GENERAL DEL PODER JUDICIAL']
['Juzgados de Instrucción']
http

for seccion in raiz.xpath('//seccion[contains(@nombre, "I. Disposiciones generales") or contains(@nombre, "III. Otras disposiciones") or contains(@nombre, "V. Anuncios. - A. Contratación del Sector Público") or contains(@nombre, "T.C. Sección del Tribunal Constitucional")]'):
#for seccion in raiz.xpath('//seccion[contains(@nombre, "I. Disposiciones generales")]'):

    nombre_seccion = seccion.xpath('@nombre')
    print(nombre_seccion)

    for departamento in seccion:
        nombre_departamento = departamento.xpath('@nombre') 

        for epigrafe in departamento:
            nombre_epigrafe = epigrafe.xpath('@nombre')

            #print(epigrafe.xpath('@id'))

            for item in epigrafe:
                item_id = item.xpath('@id')
                print(item_id)
                item_name = item.xpath('.//titulo/text()') 

                item_urlXml = "https://www.boe.es" + str(item.xpath('.//urlXml/text()'))[2:-2]

                tabla_resumen = tabla_resumen.append({'Seccion': nombre_seccion, 
                                                      'Departamento': nombre_departamento, 
                                                      'Epigrafe' : nombre_epigrafe,
                                                      'Item_id': item_id, 
                                                      'Item_Nombre' : item_name, 
                                                      'Item_URL_XML' : item_urlXml},
                                                      ignore_index=True)


In [712]:
len(tabla_resumen)

62

In [713]:
tabla_resumen['Item_id'][5:8]

5    [BOE-A-2021-3440]
6    [BOE-A-2021-3441]
7    [BOE-A-2021-3442]
Name: Item_id, dtype: object

In [714]:
tabla_resumen['Item_URL_XML'][5:8]

5    https://www.boe.es/diario_boe/xml.php?id=BOE-A...
6    https://www.boe.es/diario_boe/xml.php?id=BOE-A...
7    https://www.boe.es/diario_boe/xml.php?id=BOE-A...
Name: Item_URL_XML, dtype: object

In [715]:
tabla_resumen.sort_values('Item_id')

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Item_id,Seccion
0,"[MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTACIÓN]",[Fondos europeos agrícolas],"[Real Decreto 94/2021, de 16 de febrero, por e...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-3417],[I. Disposiciones generales]
1,[MINISTERIO PARA LA TRANSICIÓN ECOLÓGICA Y EL ...,[Combustibles. Precios],"[Resolución de 25 de febrero de 2021, de la Di...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-3418],[I. Disposiciones generales]
2,[MINISTERIO DE ASUNTOS ECONÓMICOS Y TRANSFORMA...,[Ordenación de pagos],"[Resolución de 2 de marzo de 2021, conjunta de...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-3419],[I. Disposiciones generales]
3,[MINISTERIO DE ASUNTOS ECONÓMICOS Y TRANSFORMA...,[Medidas financieras],"[Resolución de 4 de marzo de 2021, de la Secre...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-3420],[I. Disposiciones generales]
4,"[MINISTERIO DE INCLUSIÓN, SEGURIDAD SOCIAL Y M...",[Procedimientos administrativos. Gestión infor...,"[Orden ISM/189/2021, de 3 de marzo, por la que...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-3421],[I. Disposiciones generales]
...,...,...,...,...,...,...
57,"[MINISTERIO DE TRANSPORTES, MOVILIDAD Y AGENDA...",[],[Anuncio de licitación de: ADIF - Presidencia....,https://www.boe.es/diario_boe/xml.php?id=BOE-B...,[BOE-B-2021-10604],[V. Anuncios. - A. Contratación del Sector Púb...
58,[MINISTERIO DE TRABAJO Y ECONOMÍA SOCIAL],[],[Anuncio de formalización de contratos de: Dir...,https://www.boe.es/diario_boe/xml.php?id=BOE-B...,[BOE-B-2021-10605],[V. Anuncios. - A. Contratación del Sector Púb...
59,[MINISTERIO DE ASUNTOS ECONÓMICOS Y TRANSFORMA...,[],[Anuncio de formalización de contratos de: Dir...,https://www.boe.es/diario_boe/xml.php?id=BOE-B...,[BOE-B-2021-10606],[V. Anuncios. - A. Contratación del Sector Púb...
60,"[MINISTERIO DE INCLUSIÓN, SEGURIDAD SOCIAL Y M...",[],[Anuncio de formalización de contratos de: INS...,https://www.boe.es/diario_boe/xml.php?id=BOE-B...,[BOE-B-2021-10607],[V. Anuncios. - A. Contratación del Sector Púb...


In [716]:
len(tabla_resumen)

62

In [778]:
# Convierte columnas Departamento y Epigrafe a tipo str para poder realizar búsqueda de palabra Comunidad y variantes
tabla_resumen['Epigrafe'] = tabla_resumen['Epigrafe'].astype(str)
tabla_resumen['Departamento'] = tabla_resumen['Departamento'].astype(str)

In [779]:
# Elimina disposiciones con departamento o epigrafe que contenga las palabras Comunidad y sus variantes
terminos = ['Comunidad', 'Comunitat', 'COMUNIDAD', 'COMUNITAT']
tabla_resumen = tabla_resumen.drop(tabla_resumen.index[tabla_resumen['Epigrafe'].str.contains('|'.join(terminos), na=False)])
tabla_resumen = tabla_resumen.drop(tabla_resumen.index[tabla_resumen['Departamento'].str.contains('|'.join(terminos), na=False)])

In [780]:
len(tabla_resumen)

46

In [781]:
tabla_resumen[15:35]

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Item_id,Seccion
16,['MINISTERIO PARA LA TRANSICIÓN ECOLÓGICA Y EL...,['Convenios'],"[Resolución de 25 de febrero de 2021, de la Pr...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-3451],[III. Otras disposiciones]
17,['MINISTERIO DE CULTURA Y DEPORTE'],['Premios'],"[Resolución de 23 de febrero de 2021, de la Di...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-3452],[III. Otras disposiciones]
18,['MINISTERIO DE CULTURA Y DEPORTE'],['Premios'],"[Resolución de 23 de febrero de 2021, de la Di...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-3453],[III. Otras disposiciones]
19,['MINISTERIO DE CULTURA Y DEPORTE'],['Premios'],"[Resolución de 23 de febrero de 2021, de la Di...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-3454],[III. Otras disposiciones]
20,['MINISTERIO DE CULTURA Y DEPORTE'],['Premios'],"[Resolución de 23 de febrero de 2021, de la Di...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-3455],[III. Otras disposiciones]
21,['MINISTERIO DE CULTURA Y DEPORTE'],['Premios'],"[Resolución de 23 de febrero de 2021, de la Di...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-3456],[III. Otras disposiciones]
22,['MINISTERIO DE CULTURA Y DEPORTE'],['Premios'],"[Resolución de 23 de febrero de 2021, de la Di...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-3457],[III. Otras disposiciones]
23,['MINISTERIO DE CULTURA Y DEPORTE'],['Premios'],"[Resolución de 23 de febrero de 2021, de la Di...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-3458],[III. Otras disposiciones]
24,['MINISTERIO DE CULTURA Y DEPORTE'],['Premios'],"[Resolución de 23 de febrero de 2021, de la Di...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-3459],[III. Otras disposiciones]
25,['MINISTERIO DE CULTURA Y DEPORTE'],['Premios'],"[Resolución de 23 de febrero de 2021, de la Di...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-3460],[III. Otras disposiciones]


In [782]:
tabla_resumen[tabla_resumen['Epigrafe'].str.contains('|'.join(terminos), na=False)]

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Item_id,Seccion


In [783]:
len(tabla_resumen)

46

In [784]:
tabla_resumen.sort_values('Item_id')

,Departamento,Epigrafe,Item_Nombre,Item_URL_XML,Item_id,Seccion
0,"['MINISTERIO DE AGRICULTURA, PESCA Y ALIMENTAC...",['Fondos europeos agrícolas'],"[Real Decreto 94/2021, de 16 de febrero, por e...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-3417],[I. Disposiciones generales]
1,['MINISTERIO PARA LA TRANSICIÓN ECOLÓGICA Y EL...,['Combustibles. Precios'],"[Resolución de 25 de febrero de 2021, de la Di...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-3418],[I. Disposiciones generales]
2,['MINISTERIO DE ASUNTOS ECONÓMICOS Y TRANSFORM...,['Ordenación de pagos'],"[Resolución de 2 de marzo de 2021, conjunta de...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-3419],[I. Disposiciones generales]
3,['MINISTERIO DE ASUNTOS ECONÓMICOS Y TRANSFORM...,['Medidas financieras'],"[Resolución de 4 de marzo de 2021, de la Secre...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-3420],[I. Disposiciones generales]
4,"['MINISTERIO DE INCLUSIÓN, SEGURIDAD SOCIAL Y ...",['Procedimientos administrativos. Gestión info...,"[Orden ISM/189/2021, de 3 de marzo, por la que...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-3421],[I. Disposiciones generales]
5,['CONSEJO GENERAL DEL PODER JUDICIAL'],['Juzgados de Instrucción'],"[Acuerdo de 24 de febrero de 2021, de la Comis...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-3440],[III. Otras disposiciones]
6,['MINISTERIO DE DEFENSA'],['Homologaciones'],"[Resolución 320/38054/2021, de 17 de febrero, ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-3441],[III. Otras disposiciones]
7,['MINISTERIO DE DEFENSA'],['Homologaciones'],"[Resolución 320/38055/2021, de 16 de febrero, ...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-3442],[III. Otras disposiciones]
8,['MINISTERIO DE TRABAJO Y ECONOMÍA SOCIAL'],['Convenios colectivos de trabajo'],"[Resolución de 23 de febrero de 2021, de la Di...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-3443],[III. Otras disposiciones]
9,"['MINISTERIO DE INDUSTRIA, COMERCIO Y TURISMO']",['Documentación administrativa'],"[Resolución de 24 de febrero de 2021, de la Su...",https://www.boe.es/diario_boe/xml.php?id=BOE-A...,[BOE-A-2021-3444],[III. Otras disposiciones]


# Descarga ficheros XML asociados

In [723]:
for item_URL in tabla_resumen['Item_URL_XML']:
    r = requests.get(item_URL)
    #f = './BOEs/' + item_URL[-16:] + '.xml'
    ### Separa el número del BOE del resto de la cadena y aplica expresión REGEX 
    print(item_URL.split('='))
    print(re.match('BOE\-(A|B)\-[0-9]+\-[0-9]+',item_URL.split('=')[1]))
    filename = re.match('BOE\-(A|B)\-[0-9]+\-[0-9]+',item_URL.split('=')[1]).group()
    print(filename)
    f = './BOEs/' + filename + '.xml'
    save_html(r.content, f)

['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-3417']
<re.Match object; span=(0, 15), match='BOE-A-2021-3417'>
BOE-A-2021-3417
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-3418']
<re.Match object; span=(0, 15), match='BOE-A-2021-3418'>
BOE-A-2021-3418
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-3419']
<re.Match object; span=(0, 15), match='BOE-A-2021-3419'>
BOE-A-2021-3419
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-3420']
<re.Match object; span=(0, 15), match='BOE-A-2021-3420'>
BOE-A-2021-3420
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-3421']
<re.Match object; span=(0, 15), match='BOE-A-2021-3421'>
BOE-A-2021-3421
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-3440']
<re.Match object; span=(0, 15), match='BOE-A-2021-3440'>
BOE-A-2021-3440
['https://www.boe.es/diario_boe/xml.php?id', 'BOE-A-2021-3441']
<re.Match object; span=(0, 15), match='BOE-A-2021-3441'>
BOE-A-2021-3441
['https://www.boe.es/diario_boe/xml.php?i

# Importa de nuevo los XML generados

In [724]:

import glob
BOEs = glob.glob('./BOEs/BOE*.xml')
print (BOEs)

['./BOEs/BOE-B-2021-10602.xml', './BOEs/BOE-A-2021-3477.xml', './BOEs/BOE-A-2021-3463.xml', './BOEs/BOE-A-2021-3462.xml', './BOEs/BOE-A-2021-3476.xml', './BOEs/BOE-B-2021-10603.xml', './BOEs/BOE-B-2021-10601.xml', './BOEs/BOE-A-2021-3448.xml', './BOEs/BOE-A-2021-3460.xml', './BOEs/BOE-A-2021-3474.xml', './BOEs/BOE-A-2021-3475.xml', './BOEs/BOE-A-2021-3461.xml', './BOEs/BOE-A-2021-3449.xml', './BOEs/BOE-B-2021-10600.xml', './BOEs/BOE-B-2021-10604.xml', './BOEs/BOE-A-2021-3465.xml', './BOEs/BOE-A-2021-3471.xml', './BOEs/BOE-A-2021-3459.xml', './BOEs/BOE-B-2021-10599.xml', './BOEs/BOE-B-2021-10598.xml', './BOEs/BOE-A-2021-3458.xml', './BOEs/BOE-A-2021-3470.xml', './BOEs/BOE-A-2021-3464.xml', './BOEs/BOE-B-2021-10605.xml', './BOEs/BOE-B-2021-10607.xml', './BOEs/BOE-A-2021-3472.xml', './BOEs/BOE-A-2021-3466.xml', './BOEs/BOE-A-2021-3467.xml', './BOEs/BOE-A-2021-3473.xml', './BOEs/BOE-B-2021-10606.xml', './BOEs/BOE-A-2021-3417.xml', './BOEs/BOE-A-2021-3421.xml', './BOEs/BOE-A-2021-3420.xml',

# Genera DF con datos Análisis de cada XML

In [725]:
tabla_analisis = pd.DataFrame()

for BOE in BOEs:
    print (BOE)
    BOE_XML = etree.parse(BOE)
    raiz=BOE_XML.getroot()

    materias = [materia.text for materia in raiz.findall('analisis/materias/materia')]
    alertas = [alerta.text for alerta in raiz.findall('analisis/alertas/alerta')]
    
    referencias = [str(referencia.xpath('@referencia')).replace("['",'').replace("']",'') for referencia in raiz.findall('analisis/referencias/anteriores/anterior')]
    referencias_palabras = [referencia.text for referencia in raiz.findall('analisis/referencias/anteriores/anterior/palabra')]
    referencias_texto = [referencia.text for referencia in raiz.findall('analisis/referencias/anteriores/anterior/texto')]

    Item_Id = raiz.xpath(".//identificador/text()")
    Item_name = raiz.xpath('.//titulo/text()') 

    print(Item_Id)
    Fecha_Publicacion = datetime.strptime(str(raiz.xpath('.//fecha_publicacion/text()'))[2:-2],"%Y%m%d")
    print(Fecha_Publicacion)
    #Fecha_Publicacion = str(raiz.xpath('.//fecha_publicacion/text()'))[2:-2]
 
    tabla_analisis = tabla_analisis.append({'Item_id': Item_Id,
                                            'Item_Name' : Item_name,
                                            'Fecha_publicacion' : Fecha_Publicacion,
                                            'Materias' : materias,
                                            'Alertas' : alertas,
                                            'Referencias' : referencias,
                                            'Referencias_palabra' : referencias_palabras,
                                            'Referencias_texto' : referencias_texto},
                                            ignore_index=True)


./BOEs/BOE-B-2021-10602.xml
['BOE-B-2021-10602']
2021-03-05 00:00:00
./BOEs/BOE-A-2021-3477.xml
['BOE-A-2021-3477']
2021-03-05 00:00:00
./BOEs/BOE-A-2021-3463.xml
['BOE-A-2021-3463']
2021-03-05 00:00:00
./BOEs/BOE-A-2021-3462.xml
['BOE-A-2021-3462']
2021-03-05 00:00:00
./BOEs/BOE-A-2021-3476.xml
['BOE-A-2021-3476']
2021-03-05 00:00:00
./BOEs/BOE-B-2021-10603.xml
['BOE-B-2021-10603']
2021-03-05 00:00:00
./BOEs/BOE-B-2021-10601.xml
['BOE-B-2021-10601']
2021-03-05 00:00:00
./BOEs/BOE-A-2021-3448.xml
['BOE-A-2021-3448']
2021-03-05 00:00:00
./BOEs/BOE-A-2021-3460.xml
['BOE-A-2021-3460']
2021-03-05 00:00:00
./BOEs/BOE-A-2021-3474.xml
['BOE-A-2021-3474']
2021-03-05 00:00:00
./BOEs/BOE-A-2021-3475.xml
['BOE-A-2021-3475']
2021-03-05 00:00:00
./BOEs/BOE-A-2021-3461.xml
['BOE-A-2021-3461']
2021-03-05 00:00:00
./BOEs/BOE-A-2021-3449.xml
['BOE-A-2021-3449']
2021-03-05 00:00:00
./BOEs/BOE-B-2021-10600.xml
['BOE-B-2021-10600']
2021-03-05 00:00:00
./BOEs/BOE-B-2021-10604.xml
['BOE-B-2021-10604']
2021-

In [726]:
tabla_analisis.sort_values('Item_id')

,Alertas,Fecha_publicacion,Item_Name,Item_id,Materias,Referencias,Referencias_palabra,Referencias_texto
30,"[Agricultura, Ganadería y animales, Sistema fi...",2021-03-05,"[Real Decreto 94/2021, de 16 de febrero, por e...",[BOE-A-2021-3417],"[Ayudas, Comunidades Autónomas, Entidades fina...","[BOE-A-2018-4001, BOE-A-2002-800]","[MODIFICA, CITA]","[el art. 10.3 del Real Decreto 92/2018, de 2 d..."
33,"[Comercio, Energía, Sistema tributario]",2021-03-05,"[Resolución de 25 de febrero de 2021, de la Di...",[BOE-A-2021-3418],"[Baleares, Canarias, Carbón, Ceuta, Energía el...","[BOE-A-2020-9339, BOE-A-2015-8646, BOE-A-1992-...","[DE CONFORMIDAD con, DE CONFORMIDAD con, CITA]","[el art. 3 de la Orden TED/776/2020, de 4 de a..."
34,"[Organización de la Administración, Sistema fi...",2021-03-05,"[Resolución de 2 de marzo de 2021, conjunta de...",[BOE-A-2021-3419],[Administración y Contabilidad de la Hacienda ...,"[BOE-A-2016-6719, BOE-A-2002-12506, BOE-A-2010...","[DEROGA, DE CONFORMIDAD con, DECLARA la vigencia]",[salvo lo indicado de la disposición derogator...
32,[Sistema financiero],2021-03-05,"[Resolución de 4 de marzo de 2021, de la Secre...",[BOE-A-2021-3420],"[Administración Local, Comunidades Autónomas, ...",[BOE-A-2017-7814],[ACTUALIZA],[el anexo 1 de la Resolución de 4 de julio de ...
31,"[Administración electrónica, Derecho Administr...",2021-03-05,"[Orden ISM/189/2021, de 3 de marzo, por la que...",[BOE-A-2021-3421],"[Administración electrónica, Formularios admin...","[BOE-A-2013-3364, BOE-A-2015-11724, BOE-A-2015...","[DEROGA, DE CONFORMIDAD con, DE CONFORMIDAD con]","[la Orden ESS/486/2013, de 26 de marzo, la Ley..."
...,...,...,...,...,...,...,...,...
14,[],2021-03-05,[Anuncio de licitación de: ADIF - Presidencia....,[BOE-B-2021-10604],[],[],[],[]
23,[],2021-03-05,[Anuncio de formalización de contratos de: Dir...,[BOE-B-2021-10605],[],[],[],[]
29,[],2021-03-05,[Anuncio de formalización de contratos de: Dir...,[BOE-B-2021-10606],[],[],[],[]
24,[],2021-03-05,[Anuncio de formalización de contratos de: INS...,[BOE-B-2021-10607],[],[],[],[]


In [727]:
# Crea nueva columna vacía de tipo lista en tabla_analisis
tabla_analisis['Referencias_completas'] = [[] for i in range(len(tabla_analisis))]
tabla_analisis['Tags'] = [[] for i in range(len(tabla_analisis))]
tabla_analisis['Match_ASECORP_BBDD'] = [[] for i in range(len(tabla_analisis))]


In [728]:
# Define expresiones REGEX para búsqueda de leyes, decretos, etc. referenciadas anteriormente
pattern = ['Ley [0-9]+\/[0-9]+','Ley Orgánica [0-9]+\/[0-9]+','Decreto [0-9]+\/[0-9]+','Real Decreto [0-9]+\/[0-9]+','Real Decreto Legislativo [0-9]+\/[0-9]+','Real Decreto-ley [0-9]+\/[0-9]+','Orden [A-Z]+\/[0-9]+\/[0-9]+','Orden Circular [0-9]+\/[0-9]+','Reglamento \(UE\) [0-9]+\/[0-9]+', 'Reglamento de Ejeución \(UE\) [0-9]+\/[0-9]+' ,'Sentencia de [0-9]+ de [a-z]+ de [0-9]+','Sentencia [0-9]+\/[0-9]+','Orden de [0-9]+ de [a-z]+ de [0-9]+', 'Resolución de [0-9]+ de [a-z]+ de [0-9]+','Resolución [a-z]+\/[0-9]+\/[0-9]+', 'Nota de Servicio [0-9]+\/[0-9]+', 'Acuerdo multilateral M\-[0-9]+', 'Acuerdo Multilateral RID [0-9]+\/[0-9]+', 'Circular [0-9]+\/[0-9]+', 'Decisión \(UE\) [0-9]+\/[0-9]+', 'Decisión de Ejecución \(UE\) [0-9]+\/[0-9]+', 'Instrucción IS\-[0-9]+']

In [729]:
# Consolida las columnas Referencias_palabra y Referencias_texto en una única frase
for i, row in tabla_analisis.iterrows():
    for item_list in range(len(row['Referencias'])): 
        row['Referencias_completas'].append(row['Referencias_palabra'][item_list] + ' ' + row['Referencias_texto'][item_list])
        tabla_analisis['Tags'][i] = re.findall('|'.join(pattern), str(row['Referencias_texto']), flags=re.IGNORECASE)
    print(row['Referencias_completas'])

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['MODIFICA el art. 10.3 del Real Decreto 92/2018, de 2 de marzo', 'CITA Estatuto aprobado por Real Decreto 1441/2001, de 21 de diciembre']
['DEROGA la Orden ESS/486/2013, de 26 de marzo', 'DE CONFORMIDAD con la Ley General de la Seguridad Social, texto refundido aprobado por Real Decreto Legislativo 8/2015, de 30 de octubre', 'DE CONFORMIDAD con los arts. 5 y 6 de la Ley 39/2015, de 1 de octubre']
['ACTUALIZA el anexo 1 de la Resolución de 4 de julio de 2017']
['DE CONFORMIDAD con el art. 3 de la Orden TED/776/2020, de 4 de agosto', 'DE CONFORMIDAD con la disposición transitoria 3 del Real Decreto 738/2015, de 31 de julio', 'CITA Ley 38/1992, de 28 de diciembre']
['DEROGA salvo lo indicado de la disposición derogatoria única, con efectos desde el 14 de julio de 2021, la Resolución de 7 de julio de 2016', 'DE CONFORMIDAD con el apartado 7.2.e) de la Orden PRE/1576/2002, de 19 de junio', 'DECLARA la

In [730]:
# Elimina Tags duplicados
for i, row in tabla_analisis.iterrows():
    tabla_analisis['Tags'][i] = list(set(tabla_analisis['Tags'][i]))
    print(tabla_analisis['Tags'][i])

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Real Decreto 1441/2001', 'Real Decreto 92/2018']
['Orden ESS/486/2013', 'Real Decreto Legislativo 8/2015', 'Ley 39/2015']
['Resolución de 4 de julio de 2017']
['Ley 38/1992', 'Real Decreto 738/2015', 'Orden TED/776/2020']
['Orden PRE/1576/2002', 'Resolución de 30 de abril de 2010', 'Resolución de 7 de julio de 2016']
[]
[]
[]
[]
[]
['Real Decreto Legislativo 2/2015', 'Resolución de 19 de julio de 2019', 'Real Decreto 713/2010']
[]
[]
[]
[]
[]
[]
[]
[]
['Ley Orgánica 6/1985']
[]
[]
[]
[]
[]
[]
['Orden APA/858/2020']
[]
[]
[]
[]
[]


In [731]:
tabla_analisis

,Alertas,Fecha_publicacion,Item_Name,Item_id,Materias,Referencias,Referencias_palabra,Referencias_texto,Referencias_completas,Tags,Match_ASECORP_BBDD
0,[],2021-03-05,[Anuncio de licitación de: ADIF - Presidencia....,[BOE-B-2021-10602],[],[],[],[],[],[],[]
1,[],2021-03-05,"[Resolución de 23 de febrero de 2021, de la Su...",[BOE-A-2021-3477],[],[],[],[],[],[],[]
2,[],2021-03-05,"[Resolución de 23 de febrero de 2021, de la Di...",[BOE-A-2021-3463],[],[],[],[],[],[],[]
3,[],2021-03-05,"[Resolución de 23 de febrero de 2021, de la Di...",[BOE-A-2021-3462],[],[],[],[],[],[],[]
4,[],2021-03-05,"[Resolución de 23 de febrero de 2021, de la Di...",[BOE-A-2021-3476],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...
57,[],2021-03-05,"[Resolución de 23 de febrero de 2021, de la Di...",[BOE-A-2021-3453],[],[],[],[],[],[],[]
58,[],2021-03-05,"[Orden APA/191/2021, de 2 de marzo, por la que...",[BOE-A-2021-3447],[],[],[],[],[],[],[]
59,[Cultura y ocio],2021-03-05,"[Acuerdo 20/2021, de 18 de febrero, de la Junt...",[BOE-A-2021-3484],[],[],[],[],[],[],[]
60,[],2021-03-05,"[Orden APA/190/2021, de 2 de marzo, por la que...",[BOE-A-2021-3446],[],[],[],[],[],[],[]


In [732]:
texto = ''
for refs in tabla_analisis['Referencias_completas']:
    for ref in refs:
        texto = texto + ref + ' '
        print(ref)

MODIFICA el art. 10.3 del Real Decreto 92/2018, de 2 de marzo
CITA Estatuto aprobado por Real Decreto 1441/2001, de 21 de diciembre
DEROGA la Orden ESS/486/2013, de 26 de marzo
DE CONFORMIDAD con la Ley General de la Seguridad Social, texto refundido aprobado por Real Decreto Legislativo 8/2015, de 30 de octubre
DE CONFORMIDAD con los arts. 5 y 6 de la Ley 39/2015, de 1 de octubre
ACTUALIZA el anexo 1 de la Resolución de 4 de julio de 2017
DE CONFORMIDAD con el art. 3 de la Orden TED/776/2020, de 4 de agosto
DE CONFORMIDAD con la disposición transitoria 3 del Real Decreto 738/2015, de 31 de julio
CITA Ley 38/1992, de 28 de diciembre
DEROGA salvo lo indicado de la disposición derogatoria única, con efectos desde el 14 de julio de 2021, la Resolución de 7 de julio de 2016
DE CONFORMIDAD con el apartado 7.2.e) de la Orden PRE/1576/2002, de 19 de junio
DECLARA la vigencia de la disposición adicional única de la Resolución de 30 de abril de 2010
DE CONFORMIDAD con el art. 90.2 y 3 de la Ley

In [733]:
texto = ''
for i, row in tabla_analisis.iterrows():
    #unique_id = i
    print('\n' + str(row['Item_id']) + str(row['Item_Name']) +'\n')

    antecedente = 1
    for ref in row['Referencias_completas']:
        texto = texto + ref + ' '
        print('\t' + 'Antecedente ' + str(antecedente) + ': ' + ref + ' ' + str(re.findall('|'.join(pattern), ref, flags=re.IGNORECASE)))
        antecedente += 1


['BOE-B-2021-10602']['Anuncio de licitación de: ADIF - Presidencia. Objeto: Implantación de una Solución Integral para la Presentación Geográfica y Descripción de las Instalaciones de Servicio de ADIF, ADIF AV y otros Explotadores. Expediente: 4.20/41110.0142.']


['BOE-A-2021-3477']['Resolución de 23 de febrero de 2021, de la Subsecretaría, por la que se aprueba la actualización de cinco Cartas de Servicios del Instituto de Mayores y Servicios Sociales.']


['BOE-A-2021-3463']['Resolución de 23 de febrero de 2021, de la Dirección General del Libro y Fomento de la Lectura, por la que se convoca el Premio Nacional de Literatura en la modalidad de Poesía Joven "Miguel Hernández" correspondiente a 2021.']


['BOE-A-2021-3462']['Resolución de 23 de febrero de 2021, de la Dirección General del Libro y Fomento de la Lectura, por la que se convoca el Premio Nacional de Literatura en la modalidad de Poesía correspondiente a 2021.']


['BOE-A-2021-3476']['Resolución de 23 de febrero de 2021, d

In [734]:
# Aplica expresiones REGEX para búsqueda de leyes, decretos, etc. referenciadas anteriormente
regex_result = re.findall('|'.join(pattern), texto, flags=re.IGNORECASE)

In [735]:
print(regex_result)

['Real Decreto 92/2018', 'Real Decreto 1441/2001', 'Orden ESS/486/2013', 'Real Decreto Legislativo 8/2015', 'Ley 39/2015', 'Resolución de 4 de julio de 2017', 'Orden TED/776/2020', 'Real Decreto 738/2015', 'Ley 38/1992', 'Resolución de 7 de julio de 2016', 'Orden PRE/1576/2002', 'Resolución de 30 de abril de 2010', 'Real Decreto Legislativo 2/2015', 'Real Decreto 713/2010', 'Resolución de 19 de julio de 2019', 'Ley Orgánica 6/1985', 'Orden APA/858/2020']


In [736]:
## Elimina duplicados
boletin_flat_list = list(set(regex_result))

In [737]:
boletin_flat_list

['Real Decreto Legislativo 2/2015',
 'Real Decreto 92/2018',
 'Real Decreto 713/2010',
 'Orden PRE/1576/2002',
 'Orden ESS/486/2013',
 'Orden APA/858/2020',
 'Resolución de 4 de julio de 2017',
 'Ley 39/2015',
 'Resolución de 19 de julio de 2019',
 'Resolución de 7 de julio de 2016',
 'Real Decreto 738/2015',
 'Ley Orgánica 6/1985',
 'Orden TED/776/2020',
 'Ley 38/1992',
 'Resolución de 30 de abril de 2010',
 'Real Decreto Legislativo 8/2015',
 'Real Decreto 1441/2001']

## Importa BBDD ASECORP

In [738]:
ASECORP_BBDD = pd.read_csv('./ASECORP/ExportNormes_20210126.csv', delimiter=';')

In [739]:
ASECORP_BBDD

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma
0,RES SLT/2313/2020 CAT,2313,"Resolución SLT/2313/2020, de 24 de septiembre,...",CAT,Cataluña,NaN,Prevención,[2]
1,(zzz) RES 12/4/2007 AST,12/4/2007,"Resolución de 12 abril de 2007, de la Consejer...",AST,Asturias,NaN,M. Ambiente,[1]
2,A CORUÑA 10/7/2015,10/7/2015,Modificación de la Ordenanza de vertidos y ser...,GAL,Galicia,A Coruña,M. Ambiente,[1]
3,A CORUÑA 11/8/2014 (1),11/8/2014,Ordenanza municipal reguladora de los procedim...,GAL,Galicia,A Coruña,M. Ambiente,[1]
4,A CORUÑA 11/8/2014 (2),11/8/2014,Ordenanza de protección contra la contaminació...,GAL,Galicia,A Coruña,M. Ambiente,[1]
...,...,...,...,...,...,...,...,...
19152,ZUMAIA 1/3/2001,1/3/2001,Ordenanza reguladora de la gestión del ciclo i...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19153,ZUMAIA 12/11/1999,12/11/1999,Normativa complementaria de las Normas Subsidi...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19154,ZUMAIA 15/6/2012,15/6/2012,Modificación de la Ordenanza municipal de la g...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]
19155,ZUMAIA 6/2/2004,6/2/2004,Modificación de la Normativa complementaria de...,PVA,Pais Vasco,Zumaia,M. Ambiente,[1]


In [740]:
ASECORP_BBDD_BOE = ASECORP_BBDD.loc[ASECORP_BBDD['Ambito'] == 'España']

In [741]:
ASECORP_BBDD_BOE

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma
23,ACU 1/2020 ESP,1,Acuerdo Multilateral RID 1/2020 en virtud de l...,ESP,España,NaN,M. Ambiente / Prevención,[1][2]
35,ACU 11/6/2007 ESP,11/6/2007,Entrada en vigor del Acuerdo entre el Reino de...,ESP,España,NaN,M. Ambiente,[1]
57,ACU 17/10/2006 ESP,17/10/2006,Aplicación provisional del Acuerdo entre el Re...,ESP,España,NaN,M. Ambiente,[1]
71,ACU 19/11/2018 ESP,19/11/2018,Acuerdo de la Junta de Gobierno de la Confeder...,ESP,España,NaN,M. Ambiente,[1]
82,ACU 2/2020 ESP,2,Acuerdo Multilateral RID 2/2020 en virtud de l...,ESP,España,NaN,M. Ambiente / Prevención,[1][2]
...,...,...,...,...,...,...,...,...
18496,STC 86/2019 ESP,86,"Sentencia 86/2019, de 20 de junio de 2019. Rec...",ESP,España,NaN,M. Ambiente,[1]
18497,STC 87/2019 ESP,87,"Sentencia 87/2019, de 20 de junio de 2019. Rec...",ESP,España,NaN,M. Ambiente,[1]
18498,STC 88/2018 ESP,88,"Sentencia 88/2018, de 19 de julio de 2018. Con...",ESP,España,NaN,M. Ambiente / Reg. Industrial,[1][3]
18499,STC 9/3/2010 ESP,9/3/2010,"Sentencia de 9 de marzo de 2010, de la Sala Te...",ESP,España,NaN,M. Ambiente,[1]


In [742]:
# Crea nueva columna vacía de tipo lista en ASECORP_BBDD
ASECORP_BBDD_BOE['Tags'] = [[] for i in range(len(ASECORP_BBDD_BOE))]

In [743]:
titulo = []
for i, row in ASECORP_BBDD_BOE.iterrows():
    titulo.append(str(row['Titulo']) + str(re.findall('|'.join(pattern), str(row['Titulo']), flags=re.IGNORECASE)))
    ASECORP_BBDD_BOE['Tags'][i] = re.findall('|'.join(pattern), str(row['Titulo']), flags=re.IGNORECASE)

In [744]:
ASECORP_BBDD_BOE[1200:1250]

,Codigo,Numero,Titulo,CodigoAmbito,Ambito,Municipio,TiposNorma,IdTiposNorma,Tags
11629,ORD PRE/2827/2009 ESP,2827,"Orden PRE/2827/2009, de 19 de octubre, por la ...",ESP,España,NaN,M. Ambiente,[1],"[Orden PRE/2827/2009, Real Decreto 1370/2006]"
11630,ORD PRE/2843/2009 ESP,2843,"Orden PRE/2843/2009, de 19 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2843/2009, Real Decreto 2163/1994]"
11631,ORD PRE/2851/2010 ESP,2851,"Orden PRE/2851/2010, de 4 de noviembre, por la...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2851/2010, Real Decreto 2163/1994]"
11632,ORD PRE/2871/2011 ESP,2871,"Orden PRE/2871/2011, de 25 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2871/2011, Real Decreto 2163/1994]"
11633,ORD PRE/2872/2011 ESP,2872,"Orden PRE/2872/2011, de 25 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2872/2011, Real Decreto 2163/1994]"
11634,ORD PRE/29/2004 ESP,29,"Orden PRE/29/2004, de 15 de enero, por la que ...",ESP,España,NaN,M. Ambiente,[1],"[Orden PRE/29/2004, Real Decreto 837/2002]"
11635,ORD PRE/2922/2005 ESP,2922,"Orden PRE/2922/2005, de 19 de septiembre, por ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2922/2005, Orden de 8 de marzo de 1..."
11636,ORD PRE/2957/2008 ESP,2957,"Orden PRE/2957/2008, de 10 de octubre, por la ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/2957/2008, Real Decreto 2163/1994]"
11637,ORD PRE/3/2006 ESP,3,"Orden PRE/3/2006, de 12 de enero, por la que s...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/3/2006, Real Decreto 255/2003]"
11638,ORD PRE/3159/2004 ESP,3159,"Orden PRE/3159/2004, de 28 de septiembre, por ...",ESP,España,NaN,M. Ambiente / Prevención,[1][2],"[Orden PRE/3159/2004, Real Decreto 1406/1989]"


In [745]:
# titulo

In [746]:
len(titulo)

2959

In [747]:
## Busca patrones regex definidos en columna títulos del DF 
boletin = []
for i, row in ASECORP_BBDD_BOE.iterrows():
    boletin.append(str(re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE)))

In [748]:
boletin[0:25]

["['Acuerdo Multilateral RID 1/2020']",
 '[]',
 '[]',
 '[]',
 "['Acuerdo Multilateral RID 2/2020']",
 '[]',
 "['Acuerdo Multilateral RID 1/2013']",
 '[]',
 '[]',
 '[]',
 '[]',
 "['Acuerdo Multilateral RID 3/2007']",
 "['Acuerdo Multilateral RID 4/2009']",
 '[]',
 '[]',
 "['Acuerdo Multilateral M-150']",
 "['Acuerdo Multilateral M-159']",
 "['Acuerdo multilateral M-168']",
 "['Acuerdo Multilateral M-170']",
 "['Acuerdo Multilateral M-171']",
 "['Acuerdo Multilateral M-173']",
 "['Acuerdo Multilateral M-175']",
 '[]',
 "['Acuerdo Multilateral M-178']",
 "['Acuerdo Multilateral M-198']"]

In [749]:
## Para cada fila de la BBDD recoge la expresión REGEX encontrada y si no existe no la incluye 
## en la lista resultante llamada 'boletin'. Además si no detecta expresión la cuenta como vacía
## en variable 'n_vacios', y la añade a la lista 'vacios' para inspeccionar posteriormente 
boletin = []
vacios = []
n_vacios = 0
for i, row in ASECORP_BBDD_BOE.iterrows():
    regex_result = re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE)
    if len(regex_result) != 0:
        boletin.append(re.findall('|'.join(pattern), row['Titulo'], flags=re.IGNORECASE))
    else:
        n_vacios += 1
        vacios.append(row['Titulo'])

In [750]:
n_vacios

225

In [751]:
vacios[0:25]

['Entrada en vigor del Acuerdo entre el Reino de España y el Principado de Andorra sobre traslado y gestión de residuos, hecho en Madrid el 17 de octubre de 2006.',
 'Aplicación provisional del Acuerdo entre el Reino de España y el Principado de Andorra sobre traslado y gestión de residuos, hecho en Madrid el 17 de octubre de 2006.',
 'Acuerdo de la Junta de Gobierno de la Confederación Hidrográfica del Segura celebrada el día 30 de octubre de 2018 sobre aplicación de indisponibilidad o reducción de dotaciones de acuerdo al Plan Especial de Sequía y al Real Decreto que la regula',
 'Entrada en vigor del Acuerdo entre el Reino de España y el Principado de Andorra sobre el traslado de residuos, hecho en Madrid el 29 de noviembre de 2011.',
 'Canje de Notas por el que se procede a la prórroga del mantenimiento en vigor del Acuerdo entre el Reino de España y el Principado de Andorra sobre traslado y gestión de residuos, hecho en Madrid el 27 de enero de 2000.',
 'Aplicación provisional del

In [752]:
boletin[0:25]

[['Acuerdo Multilateral RID 1/2020'],
 ['Acuerdo Multilateral RID 2/2020'],
 ['Acuerdo Multilateral RID 1/2013'],
 ['Acuerdo Multilateral RID 3/2007'],
 ['Acuerdo Multilateral RID 4/2009'],
 ['Acuerdo Multilateral M-150'],
 ['Acuerdo Multilateral M-159'],
 ['Acuerdo multilateral M-168'],
 ['Acuerdo Multilateral M-170'],
 ['Acuerdo Multilateral M-171'],
 ['Acuerdo Multilateral M-173'],
 ['Acuerdo Multilateral M-175'],
 ['Acuerdo Multilateral M-178'],
 ['Acuerdo Multilateral M-198'],
 ['Acuerdo Multilateral M-214'],
 ['Acuerdo Multilateral M-218'],
 ['Acuerdo Multilateral M-226'],
 ['Acuerdo Multilateral M-228'],
 ['Acuerdo Multilateral M-231'],
 ['Acuerdo Multilateral M-236'],
 ['Acuerdo Multilateral M-245'],
 ['Acuerdo Multilateral M-254'],
 ['Acuerdo Multilateral M-259'],
 ['Acuerdo Multilateral M-265'],
 ['Acuerdo Multilateral M-268']]

In [753]:
# Flatten list of lists
boletin_ASECORP_flat_list = [item for sublist in boletin for item in sublist]

In [754]:
boletin_ASECORP_flat_list[0:25]

['Acuerdo Multilateral RID 1/2020',
 'Acuerdo Multilateral RID 2/2020',
 'Acuerdo Multilateral RID 1/2013',
 'Acuerdo Multilateral RID 3/2007',
 'Acuerdo Multilateral RID 4/2009',
 'Acuerdo Multilateral M-150',
 'Acuerdo Multilateral M-159',
 'Acuerdo multilateral M-168',
 'Acuerdo Multilateral M-170',
 'Acuerdo Multilateral M-171',
 'Acuerdo Multilateral M-173',
 'Acuerdo Multilateral M-175',
 'Acuerdo Multilateral M-178',
 'Acuerdo Multilateral M-198',
 'Acuerdo Multilateral M-214',
 'Acuerdo Multilateral M-218',
 'Acuerdo Multilateral M-226',
 'Acuerdo Multilateral M-228',
 'Acuerdo Multilateral M-231',
 'Acuerdo Multilateral M-236',
 'Acuerdo Multilateral M-245',
 'Acuerdo Multilateral M-254',
 'Acuerdo Multilateral M-259',
 'Acuerdo Multilateral M-265',
 'Acuerdo Multilateral M-268']

In [755]:
## Elimina duplicados
boletin_ASECORP_flat_list = list(set(boletin_ASECORP_flat_list))

In [756]:
boletin_ASECORP_flat_list[0:25]

['Real Decreto Legislativo 2/2011',
 'Orden PRA/375/2018',
 'Orden IET/2271/2015',
 'Orden SND/413/2020',
 'Ley 2/2013',
 'Orden FOM/1194/2011',
 'Ley 24/2001',
 'Ley 27/2006',
 'Real Decreto 920/2017',
 'Real Decreto 997/2002',
 'Orden MAM/698/2007',
 'Real Decreto 1334/2012',
 'Real Decreto 1277/2003',
 'Resolución de 7 de octubre de 2008',
 'Instrucción IS-16',
 'Orden de 3 de junio de 1986',
 'Resolución de 13 de marzo de 2009',
 'Real Decreto 311/2016',
 'Ley 1/2018',
 'Orden de 6 de Octubre de 1980',
 'Real Decreto 387/1996',
 'Real Decreto 201/2012',
 'Orden TEC/1196/2018',
 'Resolución de 7 de marzo de 2017',
 'Orden de 24 de febrero de 1993']

In [757]:
## Busca coincidencias entre lista boletines BOEs explorados y lista boletines de BBDD ASECORP
set(boletin_flat_list) & set(boletin_ASECORP_flat_list)

{'Ley 38/1992',
 'Real Decreto 738/2015',
 'Real Decreto Legislativo 2/2015',
 'Real Decreto Legislativo 8/2015'}

In [758]:
#tabla_analisis['Tags'].isin(ASECORP_BBDD_BOE['Tags'])
for row_to_compare in tabla_analisis['Tags']:
    for row_comparing in ASECORP_BBDD_BOE['Tags']:
        if set(row_comparing) & set(row_to_compare):
            print(set(row_comparing) & set(row_to_compare))

{'Real Decreto Legislativo 8/2015'}
{'Ley 38/1992'}
{'Ley 38/1992'}
{'Real Decreto 738/2015'}
{'Real Decreto Legislativo 2/2015'}


In [759]:
for i, row_to_compare in tabla_analisis.iterrows():
    for j, row_comparing in ASECORP_BBDD_BOE.iterrows():
        if set(row_to_compare['Tags']) & set(row_comparing['Tags']):
            tabla_analisis['Match_ASECORP_BBDD'][i].append (ASECORP_BBDD_BOE['Codigo'][j])
            print(str(set(row_to_compare['Tags']) & set(row_comparing['Tags'])) + ' ' + str(row_comparing['Codigo']))

{'Real Decreto Legislativo 8/2015'} RDG 8/2015 ESP
{'Ley 38/1992'} LEY 28/2014 ESP
{'Ley 38/1992'} LEY 38/1992 ESP
{'Real Decreto 738/2015'} RD 738/2015 ESP
{'Real Decreto Legislativo 2/2015'} RDG 2/2015 ESP


In [761]:
tabla_analisis

,Alertas,Fecha_publicacion,Item_Name,Item_id,Materias,Referencias,Referencias_palabra,Referencias_texto,Referencias_completas,Tags,Match_ASECORP_BBDD
0,[],2021-03-05,[Anuncio de licitación de: ADIF - Presidencia....,[BOE-B-2021-10602],[],[],[],[],[],[],[]
1,[],2021-03-05,"[Resolución de 23 de febrero de 2021, de la Su...",[BOE-A-2021-3477],[],[],[],[],[],[],[]
2,[],2021-03-05,"[Resolución de 23 de febrero de 2021, de la Di...",[BOE-A-2021-3463],[],[],[],[],[],[],[]
3,[],2021-03-05,"[Resolución de 23 de febrero de 2021, de la Di...",[BOE-A-2021-3462],[],[],[],[],[],[],[]
4,[],2021-03-05,"[Resolución de 23 de febrero de 2021, de la Di...",[BOE-A-2021-3476],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...
57,[],2021-03-05,"[Resolución de 23 de febrero de 2021, de la Di...",[BOE-A-2021-3453],[],[],[],[],[],[],[]
58,[],2021-03-05,"[Orden APA/191/2021, de 2 de marzo, por la que...",[BOE-A-2021-3447],[],[],[],[],[],[],[]
59,[Cultura y ocio],2021-03-05,"[Acuerdo 20/2021, de 18 de febrero, de la Junt...",[BOE-A-2021-3484],[],[],[],[],[],[],[]
60,[],2021-03-05,"[Orden APA/190/2021, de 2 de marzo, por la que...",[BOE-A-2021-3446],[],[],[],[],[],[],[]


In [762]:
tabla_analisis['Match_ASECORP_BBDD'][1]

[]

In [763]:
tabla_analisis['Tags'][1]

[]

# Genera Fichero EXCEL de resultados

In [764]:
## Cambia orden de columnas y elimina las no necesarias  
tabla_analisis_final = tabla_analisis[['Item_id','Item_Name','Fecha_publicacion','Materias','Alertas','Referencias','Referencias_completas','Tags','Match_ASECORP_BBDD']]

In [765]:
## Crea función que convierte lista a string en todas las columnas de tabla_analisis
## para evitar en presentación final los caracteres [' '] propios de las listas
def list2Str(lst):
    if type(lst) is list: # apply conversion to list columns
        return", ".join(lst)
    else:
        return lst

In [766]:
## Aplica función de conversión de listas a strings
tabla_analisis_final = tabla_analisis_final.apply(lambda x: [list2Str(i) for i in x])

In [767]:
  tabla_analisis_final[0:10]

,Item_id,Item_Name,Fecha_publicacion,Materias,Alertas,Referencias,Referencias_completas,Tags,Match_ASECORP_BBDD
0,BOE-B-2021-10602,Anuncio de licitación de: ADIF - Presidencia. ...,2021-03-05,,,,,,
1,BOE-A-2021-3477,"Resolución de 23 de febrero de 2021, de la Sub...",2021-03-05,,,,,,
2,BOE-A-2021-3463,"Resolución de 23 de febrero de 2021, de la Dir...",2021-03-05,,,,,,
3,BOE-A-2021-3462,"Resolución de 23 de febrero de 2021, de la Dir...",2021-03-05,,,,,,
4,BOE-A-2021-3476,"Resolución de 23 de febrero de 2021, de la Dir...",2021-03-05,,,,,,
5,BOE-B-2021-10603,Anuncio de formalización de contratos de: Pres...,2021-03-05,,,,,,
6,BOE-B-2021-10601,Anuncio de formalización de contratos de: ADIF...,2021-03-05,,,,,,
7,BOE-A-2021-3448,"Orden APA/192/2021, de 2 de marzo, por la que ...",2021-03-05,,,,,,
8,BOE-A-2021-3460,"Resolución de 23 de febrero de 2021, de la Dir...",2021-03-05,,,,,,
9,BOE-A-2021-3474,"Resolución de 25 de febrero de 2021, de la Sec...",2021-03-05,,,,,,


In [768]:
## Generar hyperlink a artículo BOE en CSV "=HYPERLINK("https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf";"BOE-A-2021-1474")"
## https://www.boe.es/diario_boe/xml.php?id=## https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf

tabla_analisis_final_CSV = tabla_analisis_final

In [769]:
# Compone y genera enlace a PDF del BOE correspondiente
tabla_analisis_final_CSV['Item_id'] = '=HIPERVINCULO(' + '"https://www.boe.es/boe/dias/' \
                                                        + tabla_analisis_final_CSV['Fecha_publicacion'].map(lambda x: x.strftime('%Y')) + '/'\
                                                        + tabla_analisis_final_CSV['Fecha_publicacion'].map(lambda x: x.strftime('%m')) + '/'\
                                                        + tabla_analisis_final_CSV['Fecha_publicacion'].map(lambda x: x.strftime('%d')) + '/'\
                                                        + 'pdfs/'\
                                                        + tabla_analisis_final_CSV['Item_id'] + '.pdf";'\
                                                        + '"' + tabla_analisis_final_CSV['Item_id'] + '")'

In [770]:
print(tabla_analisis_final_CSV['Item_id'][0])

=HIPERVINCULO("https://www.boe.es/boe/dias/2021/03/05/pdfs/BOE-B-2021-10602.pdf";"BOE-B-2021-10602")


In [771]:
# tabla_analisis_final_CSV.to_excel("./ASECORP/Resultados_Matching.xlsx", sheet_name='MATCHING_BOE') 

In [772]:
tabla_analisis_final_CSV.to_csv("./ASECORP/Resultados_Matching_BOE_" + today.strftime("%Y%m%d") + ".csv", index=False) 

# ------------------------------------------------------------------------